In [1]:
import json

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [3]:
import import_ipynb
from aux import utils
from aux import nlp
from aux import relation_extraction
from aux import defs
import preparation

importing Jupyter notebook from /Users/YK/mt/project/aux/utils.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/nlp.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/relation_extraction.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/defs.ipynb
importing Jupyter notebook from preparation.ipynb


In [4]:
def is_background(relation):
    return relation is not None and relation.type == "Background"


def has_nested_background(relation):
    return (
        relation is not None
        and
        (
            is_background(relation.left_child)
            or is_background(relation.right_child)
            or has_nested_background(relation.left_child)
            or has_nested_background(relation.right_child)
        )
    )


def test__has_nested_background():
    r010 = relation_extraction.Relation("Background", None, None, None, None)
    r011 = relation_extraction.Relation("-", None, None, None, None)
    r000 = relation_extraction.Relation("-", None, None, None, None)
    r001 = relation_extraction.Relation("-", None, None, None, None)
    r01  = relation_extraction.Relation("-", None, None, r010, r011)
    r00  = relation_extraction.Relation("-", None, None, r000, r001)
    r0   = relation_extraction.Relation("-", None, None, r00, r01)

    assert has_nested_background(r0)

    r010 = relation_extraction.Relation("-", None, None, None, None)
    r011 = relation_extraction.Relation("-", None, None, None, None)
    r000 = relation_extraction.Relation("-", None, None, None, None)
    r001 = relation_extraction.Relation("-", None, None, None, None)
    r01  = relation_extraction.Relation("-", None, None, r010, r011)
    r00  = relation_extraction.Relation("-", None, None, r000, r001)
    r0   = relation_extraction.Relation("-", None, None, r00, r01)

    assert has_nested_background(r0) is False

    r010 = relation_extraction.Relation("-", None, None, None, None)
    r011 = relation_extraction.Relation("-", None, None, None, None)
    r000 = relation_extraction.Relation("-", None, None, None, None)
    r001 = relation_extraction.Relation("-", None, None, None, None)
    r01  = relation_extraction.Relation("Background", None, None, r010, r011)
    r00  = relation_extraction.Relation("-", None, None, r000, r001)
    r0   = relation_extraction.Relation("-", None, None, r00, r01)

    assert has_nested_background(r0)

    assert has_nested_background(None) is False
    
    
test__has_nested_background()

In [5]:
class RuleExplanation06(defs.Rule):
    name = "explanation_06"
    relation_type = "Explanation"
    reasons = {
        "COMMON_PATTERN_-_ELABORATION->>BACKGROUND": 
            defs.Reason(
                1,
                "Common pattern ( -Elaboration->>Background)."
            ),
        "COMMON_PATTERN_-_JOINT->>BACKGROUND": 
            defs.Reason(
                1,
                "Common pattern ( -Joint->>Background)."
            )
    }
    
    def generate_statement(self, text, relation, verbose=False):
        assert(relation is not None and relation.type == "Explanation")
        info = preparation.Preprocessor.prepare_extended_info(text, relation, verbose)
        if info is None:
            utils.print_if_verbose("Extended info preparation wasn't successful.", verbose)
            return None

        if info.satellite_info.relation is None:
            return None
        
        utils.print_if_verbose(
                "Nucleus relation type: "
                f"'{utils.get_relation_type(info.nucleus_info.relation)}'.",
                verbose
            )
        utils.print_if_verbose(
            "Satellite relation type: "
            f"'{utils.get_relation_type(info.satellite_info.relation)}'.",
            verbose
        )
            
        reason = None
        
        assert info.satellite_info.relation is not None
        if info.nucleus_info.relation is None:
            if (
                info.satellite_info.relation.type == "Elaboration"
                    and has_nested_background(info.satellite_info.relation)
            ):
                reason = self.reasons["COMMON_PATTERN_-_ELABORATION->>BACKGROUND"]
            elif (
                info.satellite_info.relation.type == "Joint"
                    and has_nested_background(info.satellite_info.relation)
            ):
                reason = self.reasons["COMMON_PATTERN_-_JOINT->>BACKGROUND"]

        if reason is not None:
            utils.print_if_verbose(reason.explanation, verbose)
            
            prepared_nucleus_text = utils.remove_trailing_punctuation(
                utils.uppercase_first_letter(info.nucleus_preparation_result.prepared_text)
            )
            processed_s_text = nlp.remove_leading_words(
                info.satellite_preparation_result.prepared_text, verbose
            )
            prepared_s_text = utils.lowercase_first_letter(
                processed_s_text if processed_s_text is not None 
                    else info.satellite_preparation_result.prepared_text
            )
            statement_text = f"{prepared_nucleus_text}. Also, {prepared_s_text}"
            return defs.Statement(
                statement_text=statement_text,
                nucleus=prepared_nucleus_text,
                satellite_nucleus=prepared_s_text,
                left_boundary=relation.left.start,
                right_boundary=relation.right.end,
                nucleus_proximity=info.nucleus_proximity.value,
                rule=self.name,
                reason=reason
            )
        else:
            return None

In [6]:
if __name__ == "__main__" and "__file__" not in globals():
    rule = RuleExplanation06()

    with open("../parsed/race/train/middle/2708.txt.tree", "rt") as f:
        tree_text = f.read()

    text, relations = relation_extraction.read_relations(
        tree_text.replace("<s>", "").replace("<P>", "")
    )

    expl = relations["Explanation"][1]
    print(text[expl.left.start:expl.right.end])

    statement =rule.generate_statement(text, expl, verbose=True)
    print("\nRESULT:")
    if statement is not None:
        print(json.dumps(statement._asdict(), indent=2))

You can give your parents a happy day with a card or a joke .  It 's also lovely when a kid cleans up his or her room without being asked .  And if you try not fight with your brothers or sisters , your parents will be so happy .  Do your best at whatever you do .  
Nucleus is on the left.
Nucleus's depth <= 3.
Nucleus is flat.
Will use the whole segment.
Satellite's nucleus is on the left.
Nuclei proximity is NucleusProximity.NEAR
Satellite's (left) nucleus contains '.', '!', '?', or ';'.
Will use the whole segment.
Text extracted from the satellite:
It's also lovely when a kid cleans up his or her room without being asked.  And if you try not fight with your brothers or sisters, your parents will be so happy.  Do your best at whatever you do.
Nucleus relation type: '-'.
Satellite relation type: 'Joint'.
Common pattern ( -Joint->>Background).
Removing tokens before the first NP:
-- syntactic parsing result
 (ROOT
  (S
    (NP (PRP It))
    (VP (VBZ 's)
      (ADVP (RB also))
      (AD